## Dask

Dask é uma biblioteca para computação paralela em Python. É composto de duas partes:
- *Big Data collections*: estruturas de dados que utilizam Numpy, Pandas or *iterators*, aplicando-os para ambientes distribuídos e utilizando técnicas para carregamento e processamento de grandes quantidades de dados;
- *Dynamic task scheduling*: escalonador que realiza a execução paralela e distribuída das estruturas existentes em Dask.

![im](https://docs.dask.org/en/stable/_images/dask-overview.svg)
Imagem: [Dask.org](https://docs.dask.org/en/stable/)




```python
from time import sleep
```

Como abordagem inicial, definiremos duas funções. Nelas, é adicionado um tempo de processamento extra de 1 segundo.

```python
#Incremento e adição
def inc_ser(x):
    print('.', end='')
    sleep(1)
    return x+1

def add_ser(x,y):
    print('.', end='')
    sleep(1)
    return x+y
```

Definidas as funções, realizaremos o cálculo de da seguinte operação e observaremos o tempo de exeução:
```python
%%time
x = inc_ser(10)
y = inc_ser(20)
z = add_ser(x,y)
print()
```

Aqui, é esperado que tome um tempo de 3 segundos, visto que para cada incremento é aguardado 1 segundo, acrescido de 1 segundo da operação de soma.

Uma importante função no dask é o método `delayed`. Observe seu uso a seguir.

```python
import dask
from dask import delayed
```

A função `delayed` é preparada para que seja usada como um *python decorator*, podendo ser aplicada a qualquer função.

```python
#Incremento e adição realizado utilizando delayed
@delayed
def inc(x):
    print('.', end='')
    sleep(1)
    return x+1

@delayed
def add(x,y):
    print('.', end='')
    sleep(1)
    return x+y
```

Em seguida, execute a chamada às funções modificadas com o uso do `delayed`:

```python
%%time
x = inc(10)
y = inc(20)
z = add(x,y)
print()
```

Observemos que o código retorna imediamente. Isso porque o cálculo não foi realmente executado. Apenas foi criado o grafo DAG com as operações. Inclusive, podemos visualizar o grafo:

```python
z.visualize()
```

Observe que as operações `inc` estão no mesmo nível, indicando que serão executadas em paralelo. Seus resultados serão passados à operação `add`.

Precisamos explicitar que a computação será realizada, com o método `.compute()`.

```python
z.compute()
```

Agora aplicaremos estas mesmas comparações utilizando valores de uma lista.

Visulizemos o DAG gerado:
```python
soma.visualize()
```

Vamos calcular e observar o tempo necessário para o processamento.
```python
%%time
result = soma.compute()
print()
print(result)
```

### Dask client

Podemos instanciar na máquina local o escalonador Dask, ao executar `Client` sem parâmetros:

```python
from dask.distributed import Client
client = Client()
```

Após instanciar o escalonador, podemos acessar seu *dashboard* na porta `8787`. Para tal, acesse:

[http://localhost:8787](http://localhost:8787)


Observando o *dashboard* do escalonador, execute o método `.compute()`:

```python
%%time
z.compute()
```

Execute novamente teste anterior:

```python
%%time
values = list(range(10,25))
results = []
for x in values:
    y = delayed(inc)(x)
    results.append(y)
    
total = delayed(sum)(results)
total.visualize()
```

Novamente, execute o método `.compute()` observando o *dashboard*: 

```python
%%time
total.compute()
```

É possível também configurar a quantidade de *threads* que serão utilizadas para o processamento.

Primeiro, fecharemos o escalonador alocado previamente:

```python
client.close()
```

Em seguida, alocaremos um novo escalonador, definindo um novo número de *threads*:

```python
client = Client(n_workers=8)
```

Atualize a página do *dashboard*, para que sejam acessadas informações refentes à nova instância do escalonador. Após atualizar a página, calcule novamente a soma dos valores na lista e observe o tempo de execução.

```python
%%time
print(total.compute())
```

Como exemplo, execute utilizando uma quantidade maior de elementos:

```python
%%time
values = list(range(250))
results = []
for x in values:
    y = delayed(inc)(x)
    results.append(y)
    
total = delayed(sum)(results)
print(total.compute())
```

## Referências

- [Dask. Scale the Python tools you love.](https://docs.dask.org/en/stable/)  
- [BOCHMAN, D. Dask: Machine Learning & Data Science Open-source Spotlight. Youtube.](https://www.youtube.com/watch?v=Alwgx_1qsj4&t=755s)